In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
ticker = yf.Ticker("AAPL")
info = ticker.info
 
data = yf.download("IBM", start='2020-10-01', end='2023-10-01')
df = pd.DataFrame(data)
df['SMA_10'] = df['Close'].rolling(window=10).mean()
df.dropna(subset=['SMA_10'], inplace=True)
df.reset_index(inplace=True)

In [ ]:
# Defino dos decimales para los datos flotantes
pd.options.display.float_format = '{:.2f}'.format
df.head(15)

In [ ]:
df.corr()

In [ ]:
# Calculo el Rango Medio Verdadero. Índice para saber la volatilidad de una acción
periodo_atr = 14
df['Rango'] = df['High'] - df['Low']
df['TR'] = df[['High', 'Close']].max(axis=1) - df[['Low', 'Close']].min(axis=1)
df['ATR'] = df['TR'].rolling(window=periodo_atr).mean()

df.iloc[30]

In [ ]:
# Preparo los datos de la siguiente manera: 30 días consecutivos serán de entrada (data_in), y el día posterior será la salida (data_out).
time_step = 30
data_in = []
data_out = []

for i in range(time_step, len(df)-time_step):
    data_in.append(df[i-time_step: i])
    data_out.append(df.iloc[i])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Paso 1: Preparar los datos
data_in_df = pd.concat(data_in, axis=0, ignore_index=True)
data_out_df = pd.DataFrame(data_out)

print(data_in_df.shape)
print(data_out_df.shape)

In [ ]:
# Paso 2: Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(data_in, data_out, test_size=0.2, random_state=1992)

# Paso 3: Crear y entrenar el modelo
model = LinearRegression()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}')
print(f'R2: {r2}')